In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import shutil

import tensorflow as tf

from tensorflow.keras.models import Sequential, load_model, Model
from tensorflow.keras.layers import Dense, Flatten, Input, concatenate
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dropout, MaxPooling2D
from tensorflow.keras.optimizers import SGD, Adam, RMSprop

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
import tensorflow.keras.utils as utils

from datetime import datetime

from tensorflow.keras.preprocessing.image import ImageDataGenerator

from google.colab import drive
import zipfile

## 카테고리별 학습 및 훈련데이터 나누기

In [2]:
drive.mount('/content/drive')

zip_path = '/content/drive/MyDrive/Colab Notebooks/car_damage_image.zip'

re_path = '/content/damage'

with zipfile.ZipFile(zip_path, 'r') as target_file:
    for file in target_file.namelist():
        file_path = os.path.join(re_path, file)
        if not os.path.exists(file_path):  # 이미 파일이 존재하는 경우 건너뛰기
            target_file.extract(file, re_path)

data_dir = '/content/damage/'
# train_dir = os.path.join(re_path, 'train')
test_dir = os.path.join(re_path, 'test')

damaged = ['Scratched', 'Crush', 'Breakaged', 'Separated']

# 훈련 데이터 테스트 데이터 비율 설정
train_ratio = 0.8
test_ratio = 0.2

# 각각 데미지 별 이미지 파일을 훈련데이터와 테스트데이터로 분할
for damage in damaged:
    damage_path = os.path.join(data_dir, damage) # 각 데미지 폴더 경로
    image_files = os.listdir(damage_path) # 데미지 폴더 내 이미지 파일 리스트

    train_size = int(len(image_files) * train_ratio)
    train_files = image_files[:train_size]
    test_files = image_files[train_size:]

    # 훈련데이터 폴더로 이동
    # for train_file in train_files:
    #     src_path = os.path.join(damage_path, train_file)
    #     dst_path = os.path.join(train_dir, damage)
    #     os.makedirs(dst_path, exist_ok=True) # exist_ok : 폴더가 존재하지 않으면 생성하고, 존재하는 경우에는 아무것도 만들지 않음
    #     shutil.copy(src_path, dst_path)

    # 테스트데이터 폴더로 이동
    for test_file in test_files:
        src_path = os.path.join(damage_path, test_file)
        dst_path = os.path.join(test_dir, damage)
        os.makedirs(dst_path, exist_ok=True) # exist_ok : 폴더가 존재하지 않으면 생성하고, 존재하는 경우에는 아무것도 만들지 않음
        shutil.move(src_path, dst_path)

Mounted at /content/drive


In [8]:
len('/content/damage/Breakaged/')

26

In [3]:
data_dir = '/content/damage'

# 압축 파일 생성 경로
zip_path = '/content/drive/MyDrive/Colab Notebooks/damage.zip'

# 데이터 폴더를 압축
with zipfile.ZipFile(zip_path, 'w') as new_zip:
    for root, dirs, files in os.walk(data_dir):
        for file in files:
            file_path = os.path.join(root, file)
            new_zip.write(file_path, os.path.relpath(file_path, data_dir))

print("새로운 zip 파일이 생성되었습니다:", zip_path)

새로운 zip 파일이 생성되었습니다: /content/drive/MyDrive/Colab Notebooks/damage.zip


# 차량 손상 유형 CNN 모델 구축


## ImageDataGenerator 요소
- shear_range : 수치는 시계반대방향으로 밀림강도를 radian으로 나타냄
- zoom_range : 지정된 확대/축소 범위내에 임의로 원본이미지를 확대/축소
- horizontal_flip : 수평방향 뒤집기
- rescale : 크기조정

In [ ]:
train_data = 'train_data경로'
test_data = 'test_data경로'

input_shape = (64,64)

# 데이터 전처리
train_gen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_gen = ImageDataGenerator(rescale=1./255)

# 학습 및 테스트 데이터셋 로드
train_dataset = train_gen.flow_from_directory(train_data, target_size=input_shape, batch_size=32, class_mode='categorical')
test_dataset = test_gen.flow_from_directory(test_data, target_size=input_shape, batch_size=32, class_mode='categorical')

## 모델 구성
- C P C P F D D

In [ ]:
# 모델 구성
model = Sequential()

model.add(Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=(input_shape[0], input_shape[1],3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(64, activation='relu'))
model.add(Dense(4, activation='softmax'))

# 모델 컴파일
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 모델 학습
start_time = datetime.now()

model.fit(train_dataset, epochs=30, validation_split=0.2, batch_size=50)

end_time = datetime.now()

print('걸린시간:', end_time - start_time)

# 모델 평가
test_loss, test_accuracy = model.evaluate(test_dataset)
print(f'Test Loss : {test_loss}')
print(f'Test Accuracy : {test_accuracy}')